## 智能交易
交易指的是购买和出售金融工具，如股票、债券、货币、商品或衍生品，目的是为了获利。期刊交易者整天都在查看不同的仪表板，试图找出是否买入或卖出的那一个完美决定。 这个自动化脚本可以帮助交易者和投资者很好地了解你想投资的任何股票。它使用Python的Prophet库，基于从雅虎财经获取的历史股票数据来预测近期的股票价格。

### 1. 导入Streamlit库用于创建Web应用

In [20]:
import streamlit as st

# 导入date用于处理日期
from datetime import date

# 导入yfinance库，用于从Yahoo Finance获取股票数据
import yfinance as yf

# 导入Prophet库，用于进行时间序列预测
from prophet import Prophet

# 导入Prophet的Plotly绘图接口和Plotly图形对象
from prophet.plot import plot_plotly
from plotly import graph_objs as go

### 2. 设置预测的起始日期和当前日期

In [21]:
START = "2024-01-01"
TODAY = date.today().strftime("%Y-%m-%d")

### 3. 使用Streamlit创建应用标题

In [22]:
st.title('StockForecast App')
# 设置一个选择框，让用户选择股票
stocks = ('MSFT', "TSLA", 'GOOG', 'AAPL', "NVDA")
selected_stock = st.selectbox('Select dataset for prediction', stocks)

### 4. 创建一个滑动条，让用户选择预测的年数，范围从1到4年

In [23]:
n_years = st.slider('Years of prediction:', 1, 4)
period = n_years * 365

### 5. 定义一个缓存函数，用于加载股票数据，避免每次会话重新加载

In [24]:
@st.cache_data
def load_data(ticker):
	data = yf.download(ticker, START, TODAY)  # 从Yahoo Finance下载股票数据
	data.reset_index(inplace=True)  # 重置DataFrame的索引
	return data

# 显示数据加载状态
data_load_state = st.text('Loading data...')
data = load_data(selected_stock)  # 调用函数加载数据
data_load_state.text('Loading data... done!')  # 更新加载状态

# 显示原始数据的最后几行
st.subheader('Raw data')
st.write(data.tail())

2024-05-10 17:39:01.862 No runtime found, using MemoryCacheStorageManager
2024-05-10 17:39:01.863 No runtime found, using MemoryCacheStorageManager
[*********************100%%**********************]  1 of 1 completed


### 6. 定义函数，绘制原始数据的开盘和收盘价格

In [27]:
def plot_raw_data():
	fig = go.Figure()
	fig.add_trace(go.Scatter(x=data['Date'], y=data['Open'], name="stock_open"))
	fig.add_trace(go.Scatter(x=data['Date'], y=data['Close'], name="stock_close"))
	fig.layout.update(title_text='Time Series data with Rangeslider', xaxis_rangeslider_visible=True)
	st.plotly_chart(fig)

plot_raw_data()

### 7. 使用Prophet进行预测

In [26]:
df_train = data[['Date','Close']]
df_train = df_train.rename(columns={"Date": "ds", "Close": "y"})
m = Prophet()
m.fit(df_train)

future = m.make_future_dataframe(periods=period)
forecast = m.predict(future)

# 显示并绘制预测数据
st.subheader('Forecast data')
st.write(forecast.tail())
st.write(f'Forecast plot for {n_years} years')

fig1 = plot_plotly(m, forecast)
st.plotly_chart(fig1)

# 显示预测组件
st.write("Forecast components")
fig2 = m.plot_components(forecast)
st.write(fig2)

17:39:04 - cmdstanpy - INFO - Chain [1] start processing
17:39:05 - cmdstanpy - INFO - Chain [1] done processing
/Users/heyachao/anaconda3/lib/python3.11/site-packages/prophet/plot.py:228: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

